In [1]:
import scrapetube
import pandas as pd

In [4]:
english_partial_titles = ["Left Right & Centre", "The Big Fight", "Newshour Debate", "Newstrack", "Arnab Debates", "The Debate With Arnab", "Viewpoint With Bhupendra Chaubey", "Urban Debate"]

In [3]:
column_names = ['video_id','partial_title']
english_videos_df = pd.DataFrame(columns = column_names)

In [5]:
for title in english_partial_titles:
    videos = scrapetube.get_search(query = title,limit = 300)
    # Extract video ids and create a DataFrame
    video_ids = [video['videoId'] for video in videos]
    partial_df = pd.DataFrame({'video_id': video_ids, 'partial_title': title})

    # Append the current DataFrame to the main DataFrame
    english_videos_df = pd.concat([english_videos_df, partial_df], ignore_index=True)
english_videos_df.head()

,video_id,partial_title
0,at-T2PJQgOg,Left Right & Centre
1,a7GITgqwDVg,Left Right & Centre
2,ney291VKSJM,Left Right & Centre
3,WGhcYYzEmxc,Left Right & Centre
4,2R_p9EHZKXs,Left Right & Centre


In [7]:
print(english_videos_df)

         video_id        partial_title
0     at-T2PJQgOg  Left Right & Centre
1     a7GITgqwDVg  Left Right & Centre
2     ney291VKSJM  Left Right & Centre
3     WGhcYYzEmxc  Left Right & Centre
4     2R_p9EHZKXs  Left Right & Centre
...           ...                  ...
2395  LSv5daQs2Oo         Urban Debate
2396  sSxejrqUkY0         Urban Debate
2397  IZQQWqdmklQ         Urban Debate
2398  IX03mWUoXY8         Urban Debate
2399  EQD_6F7EMQY         Urban Debate

[2400 rows x 2 columns]


In [12]:
import os
import googleapiclient.discovery

# Set your YouTube API key as an environment variable
os.environ["YOUTUBE_API_KEY"] = "API KEY" #Enter your api key here

# List of channel names to check
channels_to_check = ["MIRROR NOW", "CNN-News18", "India Today", "TIMES NOW", "Republic World" ,"NDTV"]  # Add your channel names here

# Initialize the YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=os.environ["YOUTUBE_API_KEY"])

In [13]:
data = []

# Iterate through each row in the DataFrame
for index, row in english_videos_df.iterrows():
    video_id = row["video_id"]
    show_name = row["partial_title"]

    # Use the YouTube Data API to get video details
    video_response = youtube.videos().list(part="snippet", id=video_id).execute()

    # Extract the channelId from the video details
    channel_id = video_response["items"][0]["snippet"]["channelId"]

    # Use the YouTube Data API to get channel details
    channel_response = youtube.channels().list(part="snippet", id=channel_id).execute()
    
    if "items" in channel_response:
        # Extract the channel title (name)
        channel_name = channel_response["items"][0]["snippet"]["title"]

        # Check if the channel name is in the list of channels to check
        if channel_name in channels_to_check:
            # Append data to the list
            data.append({"VideoId": video_id, "Channel": channel_name, "ShowName": show_name})

# Create a new DataFrame from the list of data
result_df = pd.DataFrame(data)

# Print or save the resulting DataFrame
print(result_df)


          VideoId     Channel             ShowName
0     Lxx28NJMhV4        NDTV  Left Right & Centre
1     G21_Qu9cHmM        NDTV  Left Right & Centre
2     Rc-kwHE2ZGo        NDTV        The Big Fight
3     PRF7yRsBG48        NDTV        The Big Fight
4     sJDYgQBezdc        NDTV        The Big Fight
...           ...         ...                  ...
1419  CWIk7CfgN0Y  MIRROR NOW         Urban Debate
1420  cXFMgqcfjYo  MIRROR NOW         Urban Debate
1421  TdtXFbM8o7s  MIRROR NOW         Urban Debate
1422  d-cSXtVYOK0  MIRROR NOW         Urban Debate
1423  sSxejrqUkY0  MIRROR NOW         Urban Debate

[1424 rows x 3 columns]


In [14]:
result_df.head()

,VideoId,Channel,ShowName
0,Lxx28NJMhV4,NDTV,Left Right & Centre
1,G21_Qu9cHmM,NDTV,Left Right & Centre
2,Rc-kwHE2ZGo,NDTV,The Big Fight
3,PRF7yRsBG48,NDTV,The Big Fight
4,sJDYgQBezdc,NDTV,The Big Fight


In [15]:
result_df.to_excel("English_channels_videos.xlsx", index=False)